# simple linear regression

In [126]:
# import base packages

import numpy as np
import pandas as pd
#import modin.pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

#import ray
#ray.init()
# # Modin will connect to the existing Ray environment
# import modin.pandas as pd


In [127]:
Z = pd.read_csv('for_modeling.csv', low_memory=False)

In [128]:
Z.head()

,op_unique_carrier,origin,dest,dep_delay,arr_delay,cancellation_code,air_time,distance,unique_fl,speed,dep_status,arr_status,mean_arr_delay,month_sin,month_cos,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,dep_hour_sin,dep_hour_cos,arr_hour_sin,arr_hour_cos
0,1222589,417043,18829,2.302585,0.000000,11836974,4.672829,6.739337,1865,477.735849,2269876,10138922,0.700079,1.000000e+00,6.123234e-17,0.000000,1.0,-0.848644,0.528964,-0.887885,0.460065,-0.730836,0.682553
1,452851,251909,147969,3.258097,1.609438,11836974,5.533389,7.684324,568,517.380952,2269876,1666388,1.286303,-5.000000e-01,-8.660254e-01,0.866025,-0.5,-0.485302,-0.874347,-0.519584,0.854419,0.979084,0.203456
2,1480677,595684,40093,0.000000,0.000000,11836974,3.784190,5.560682,5391,361.395349,9571927,10138922,0.000000,1.000000e+00,6.123234e-17,-0.866025,-0.5,0.394356,0.918958,-0.136167,-0.990686,-0.398401,-0.917211
3,471077,9251,597024,0.000000,0.000000,11836974,3.583519,4.875197,959,222.857143,9571927,10138922,0.482992,1.224647e-16,-1.000000e+00,0.866025,0.5,-0.937752,0.347305,0.997669,-0.068242,0.942261,-0.334880
4,381866,78884,194668,0.000000,0.000000,11836974,4.605170,6.501290,1127,403.030303,9571927,10138922,0.414163,5.000000e-01,-8.660254e-01,0.866025,-0.5,0.101168,-0.994869,0.997669,-0.068242,0.631088,-0.775711


In [129]:
Z.shape

(12071082, 23)

# scaling

In [130]:
Z1 = Z.copy()

In [131]:
Z1 = Z1.dropna()

In [132]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(Z1)

Z1 = pd.DataFrame(scaler.transform(Z1), index=Z1.index, columns=Z1.columns)

In [133]:
Z1.head()

,op_unique_carrier,origin,dest,dep_delay,arr_delay,cancellation_code,air_time,distance,unique_fl,speed,dep_status,arr_status,mean_arr_delay,month_sin,month_cos,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,dep_hour_sin,dep_hour_cos,arr_hour_sin,arr_hour_cos
0,0.295984,1.126201,-1.128602,2.131617,-0.405628,0.140633,0.281963,0.482075,-0.186554,1.042920,-1.888011,0.435738,0.348258,1.437215,0.019313,-0.027535,1.154550,-1.187652,0.779353,-1.106601,1.356446,-0.774938,1.297523
1,-0.871211,0.191195,-0.400112,3.210225,1.971999,0.140633,1.719586,1.707123,-0.861619,1.587918,-1.888011,-2.225992,1.710525,-0.677368,-1.210059,1.303723,-0.854492,-0.679452,-1.228281,-0.610684,2.035814,1.886628,0.609132
2,0.687337,2.137685,-1.008650,-0.467606,-0.405628,0.140633,-1.202566,-1.045891,1.648666,-0.556404,0.508990,0.435738,-1.278587,1.437215,0.019313,-1.358794,-0.854492,0.550908,1.337295,-0.094414,-1.142816,-0.257488,-1.001100
3,-0.843573,-1.182760,2.133046,-0.467606,-0.405628,0.140633,-1.537799,-1.934528,-0.658111,-2.460878,0.508990,0.435738,-0.156209,0.027493,-1.400244,1.303723,0.484870,-1.312285,0.519465,1.432291,0.446311,1.829311,-0.164376
4,-0.978849,-0.788491,-0.136679,-0.467606,-0.405628,0.140633,0.168935,0.173479,-0.570670,0.015949,0.508990,0.435738,-0.316156,0.732354,-1.210059,1.303723,-0.854492,0.140832,-1.400706,1.432291,0.446311,1.344957,-0.797785


In [134]:
Z1.columns

Index(['op_unique_carrier', 'origin', 'dest', 'dep_delay', 'arr_delay',
       'cancellation_code', 'air_time', 'distance', 'unique_fl', 'speed',
       'dep_status', 'arr_status', 'mean_arr_delay', 'month_sin', 'month_cos',
       'day_of_week_sin', 'day_of_week_cos', 'day_of_month_sin',
       'day_of_month_cos', 'dep_hour_sin', 'dep_hour_cos', 'arr_hour_sin',
       'arr_hour_cos'],
      dtype='object')

In [137]:
Z1.shape

(12071079, 23)

# linear regression model

### split dataset to X and Y

In [138]:
Y = Z1.arr_delay

In [139]:
Y.shape

(12071079,)

In [141]:
X = Z1.drop(['arr_delay', 'cancellation_code', 'dep_delay'], axis=1)

In [142]:
X.shape

(12071079, 20)

In [143]:
X.columns

Index(['op_unique_carrier', 'origin', 'dest', 'air_time', 'distance',
       'unique_fl', 'speed', 'dep_status', 'arr_status', 'mean_arr_delay',
       'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos',
       'day_of_month_sin', 'day_of_month_cos', 'dep_hour_sin', 'dep_hour_cos',
       'arr_hour_sin', 'arr_hour_cos'],
      dtype='object')

## split dataset

In [144]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

## check data dimensions

In [145]:
X_train.shape, Y_train.shape

((9656863, 20), (9656863,))

In [146]:
X_test.shape, Y_test.shape

((2414216, 20), (2414216,))

## build linear regression model

In [147]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [148]:
model = linear_model.LinearRegression()

In [149]:
model.fit(X_train, Y_train)

LinearRegression()

In [150]:
Y_pred = model.predict(X_test)

## prediction results

In [151]:
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(Y_test, Y_pred))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(Y_test, Y_pred))

Coefficients: [-9.82298089e-03 -2.57142395e-03  1.57673247e-03 -4.04466048e-02
  5.81981163e-02 -6.92533533e-03 -1.93300369e-02 -2.97968411e-02
 -8.96879259e-01  7.94213718e-02 -6.22898807e-04  4.41466131e-04
  7.19118513e-04 -7.87101557e-04  2.95209055e-04 -4.17854568e-04
  1.09319856e-03 -2.45417827e-03  3.54934258e-04  7.00225301e-03]
Intercept: 3.6626979611514667e-05
Mean squared error (MSE): 0.13
Coefficient of determination (R^2): 0.87


In [152]:
X.columns

Index(['op_unique_carrier', 'origin', 'dest', 'air_time', 'distance',
       'unique_fl', 'speed', 'dep_status', 'arr_status', 'mean_arr_delay',
       'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos',
       'day_of_month_sin', 'day_of_month_cos', 'dep_hour_sin', 'dep_hour_cos',
       'arr_hour_sin', 'arr_hour_cos'],
      dtype='object')